# Set up

In [1]:
# !git clone https://github.com/NLP-Reichman/2025_assignment_1.git
# !mv 2025_assignment_1/data data
# !rm 2025_assignment_1/ -r

# Introduction
In this assignment you will be creating tools for learning and testing language models. The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.
The relevant files are under the data folder:

- en.csv (or the equivalent JSON file)
- es.csv (or the equivalent JSON file)
- fr.csv (or the equivalent JSON file)
- in.csv (or the equivalent JSON file)
- it.csv (or the equivalent JSON file)
- nl.csv (or the equivalent JSON file)
- pt.csv (or the equivalent JSON file)
- tl.csv (or the equivalent JSON file)

In [2]:
import os
import json
# from google.colab import files
import pandas as pd
import numpy as np
from itertools import product
from collections import defaultdict, Counter
import math
from tqdm import tqdm
import random


# Implementation

In [3]:
DIR = 'data/'
START_TOKEN = '↠'
END_TOKEN = '↞'
UNK_TOKEN   = '<unk>'

## Part 1
Implement the function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. Our token definition is a single UTF-8 encoded character. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data. The vocabulary should include the `<start>` and  `<end>` tokens.

Note - do NOT lowecase the sentences.

In [ ]:
def preprocess() -> list[str]:
  '''
  Return a list of characters, representing the shared vocabulary of all languages
  '''
  vocab = []
  ########################################

  vocab: set[str] = set()
  for file in os.listdir(DIR):
      if file.endswith('.csv'):
          df = pd.read_csv(os.path.join(DIR, file))
          # Remove duplicates
          df = df.drop_duplicates(subset=['tweet_text'])
          # Remove empty tweets
          df = df[df['tweet_text'].notna()]
          
          for tweet in df['tweet_text'].astype(str):
              vocab.update(tweet)
              
  # Add special tokens
  vocab |= {START_TOKEN, END_TOKEN}
  vocab = list(vocab)
  ########################################
  return vocab

In [5]:
vocab = preprocess()
print(f"vocab length: {len(vocab)}")
print(f"Some characters in the vocab: {vocab[:10]}")

vocab length: 1805
Some characters in the vocab: ['w', '🤛', '\u3000', '尔', '🔱', '🍬', 'ｓ', '➊', '🔛', '︎']


## Part 2
Implement the function *build_lm* that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant *n*-1 sequences, and the values are dictionaries with the *n*_th tokens and their corresponding probabilities to occur. To ensure consistent probabilities calculation, please add n-1 `<start>` tokens to the beginning of a tweet and one `<end>` token at the end. For example, for a trigram model (tokens are characters), it should look something like:

{ "ab":{"c":0.5, "b":0.25, "d":0.25}, "ca":{"a":0.2, "b":0.7, "d":0.1} }

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

Please add the `<unk>` token with $p(<unk>)=1|V|$ to the LM if buiulding a smoothed LM.

In [6]:

def build_lm(lang: str, n: int, smoothed: bool = False) -> dict[str, dict[str, float]]:
    '''
    Return a language model for the given lang and n_gram (n)
    :param lang: the language of the model
    :param n: the n_gram value
    :param smoothed: boolean indicating whether to apply smoothing
    :return: a dictionary where the keys are n_grams and the values are dictionaries
    '''
    LM = {}
    ########################################
    assert n > 0, "n must be greater than 0"
    V = set(vocab)   
    vocab_length = len(vocab)
    counts = defaultdict(Counter)

    df = pd.read_csv(os.path.join(DIR, lang + '.csv'))
    # Remove empty tweets
    df = df[df['tweet_text'].notna()]
    
    # 1) Count
    for tweet in df['tweet_text']:
        seq = [START_TOKEN]*(n-1) + list(tweet) + [END_TOKEN]
        for i in range(len(seq) - n + 1):
            ctx = "".join(seq[i : i+n-1])
            nxt = seq[i+n-1]
            if nxt not in V:
                nxt = UNK_TOKEN
            counts[ctx][nxt] += 1

    # 2) convert to probabilities (+ optional Laplace) -----------------
    LM = {}
    for ctx, counter in counts.items():
        total = sum(counter.values()) + (vocab_length if smoothed else 0)
        probs = {}

        for ch in vocab:  # iterate over *all* symbols for smoothing
            cnt = counter.get(ch, 0) + (1 if smoothed else 0)
            probs[ch] = cnt / total
        LM[ctx] = probs

    # 3) uniform fallback for unseen contexts (helps eval / generate)
    LM.setdefault(
        "",
        {ch: 1.0 / vocab_length for ch in vocab}      # uniform distribution
    )

    ########################################
    return LM

In [7]:
LM = build_lm("en", 3, False)
print(f"English Language Model with 3-gram is of length: {len(LM)}")

English Language Model with 3-gram is of length: 8239


## Part 3
Implement the function *eval* that returns the perplexity of a model (dictionary) running over the data file of the given target language.

The `<unk>` should be used for unknown contexts when calculating the perplexities.

In [17]:
def eval(model: dict, target_lang: str, n: int) -> float:
    '''
    Return the perplexity value calculated over applying the model on the text file
    of the target_lang language.
    :param model: the language model
    :param target_lang: the target language
    :param n: The n-gram of the model
    :return: the perplexity value
    '''
    pp = 0
    ########################################

    path   = os.path.join(DIR, f"{target_lang}.csv")
    tweets = pd.read_csv(path)["tweet_text"].dropna().astype(str)

    # shared constants
    vocab   = list(next(iter(model.values())).keys())
    V       = len(vocab)
    uniform = 1.0 / V

    log_sum   = 0.0     # total negative-log-likelihood
    token_sum = 0       # total # predicted characters

    for tweet in tweets:
        seq = [START_TOKEN] * (n - 1) + list(tweet) + [END_TOKEN]

        for i in range(n - 1, len(seq)):
            ctx = "" if n == 1 else "".join(seq[i - n + 1 : i])
            tok = seq[i] if seq[i] in vocab else UNK_TOKEN

            prob = model.get(ctx, {}).get(tok, uniform)
            if not prob:                            # covers None or 0.0
                prob = uniform                      # 1 / |vocab|
            log_sum   -= math.log(prob)         # accumulate −ln p
            token_sum += 1

    pp = math.exp(log_sum / token_sum) if token_sum else float("inf")
    # the class lecture showed that the perplexity is defined with 2 ** H, but the correct one is using e ** H

    ########################################
    return pp


In [9]:
LM = build_lm("en", 3, True)

In [18]:
print("Perplexity of the English 3-gram model on datasets:")
print(f"On English: {eval(LM, 'en', 3): .2f}")
print(f"On French: {eval(LM, 'fr', 3): .2f}")
print(f"On Dutch: {eval(LM, 'nl', 3): .2f}")
print(f"On Tagalog: {eval(LM, 'tl', 3): .2f}")


Perplexity of the English 3-gram model on datasets:
On English:  26.75
On French:  58.51
On Dutch:  61.43
On Tagalog:  70.61


In [11]:
lm1 = build_lm("en", 1, True)
lm2 = build_lm("en", 2, True)
lm3 = build_lm("en", 3, True)
lm4 = build_lm("en", 4, True)

print("Perplexity on differnet n-gram models on English")
print(f"On 1-gram: {eval(lm1, 'en', 1): .2f}")
print(f"On 2-gram: {eval(lm2, 'en', 2): .2f}")
print(f"On 3-gram: {eval(lm3, 'en', 3): .2f}")
print(f"On 4-gram: {eval(lm4, 'en', 4): .2f}")

Perplexity on differnet n-gram models on English
On 1-gram:  37.86
On 2-gram:  21.30
On 3-gram:  26.75
On 4-gram:  59.90


## Part 4
Implement the *match* function that calls *eval* using a specific value of *n* for every possible language pair among the languages we have data for. You should call *eval* for every language pair four times, with each call assign a different value for *n* (1-4). Each language pair is composed of the source language and the target language. Before you make the call, you need to call the *lm* function to create the language model for the source language. Then you can call *eval* with the language model and the target language. The function should return a pandas DataFrame with the following four columns: *source_lang*, *target_lang*, *n*, *perplexity*. The values for the first two columns are the two-letter language codes. The value for *n* is the *n* you use for generating the specific perplexity values which you should store in the forth column.

In [12]:
languages = ['en', 'es', 'fr', 'in', 'it', 'nl', 'pt', 'tl']

In [ ]:
def match() -> pd.DataFrame:
    '''
    Return a DataFrame containing one line per every language pair and n_gram.
    Each line will contain the perplexity calculated when applying the language model
    of the source language on the text of the target language.
    :return: a DataFrame containing the perplexity values
    '''
    df  = pd.DataFrame()
    ########################################
    rows = []
    langs = [os.path.splitext(f)[0] for f in os.listdir(DIR) if f.endswith('.csv')]
    
    # ── pre-compute all 4-gram LMs once per (src, n) ──────────────────
    lm_cache = {}
    total_models = len(langs) * 4                # one model per (src, n)
    for src, n in tqdm(product(langs, range(1, 5)),
                    total=total_models,
                    desc="Building language models"):
        lm_cache[(src, n)] = build_lm(src, n, False)   # unsmoothed

    # ── evaluate every (src, tgt, n) combination ──────────────────────
    rows  = []
    total = len(langs) * len(langs) * 4
    for src, tgt, n in tqdm(product(langs, langs, range(1, 5)),
                            total=total, desc="Perplexity evaluations"):
        pp = eval(lm_cache[(src, n)], tgt, n)
        rows.append({
            "source_lang": src,
            "target_lang": tgt,
            "n":           n,
            "perplexity":  pp
        })

    df = pd.DataFrame(rows,
                        columns=["source", "target", "n", "perplexity"])
    ########################################
    return df
  
df = match()

Perplexity evaluations: 100%|██████████| 256/256 [30:21<00:00,  7.12s/it]


## Part 5
Implement the *generate* function which takes a language code, *n*, the prompt (the starting text), the number of tokens to generate, and *r*, which is the random seed for any randomized action you plan to take in your implementation. The function should start generating tokens, one by one, using the language model of the given source language and *n*. The prompt should be used as a starting point for aligning on the probabilities to be used for generating the next token.

Note - The generation of the next token should be from the LM's distribution with NO smoothing.

In [20]:
def generate(lang: str, n: int, prompt: str, number_of_tokens: int, r: int) -> str:
    '''
    Generate text in the given language using the given parameters.
    :param lang: the language of the model
    :param n: the n_gram value
    :param prompt: the prompt to start the generation
    :param number_of_tokens: the number of tokens to generate
    :param r: the random seed to use
    '''
    text = ""
    ########################################
    random.seed(r)                       # 1) reproducible RNG

    lm = build_lm(lang, n, False)        # 2) unsmoothed n-gram LM
    unigram = build_lm(lang, 1, False)['']  # 1-gram back-off

    generated = list(prompt)             # char buffer

    for _ in range(number_of_tokens):
        # 3) context key
        ctx = "" if n == 1 else "".join(generated[-(n - 1):])

        # 4) choose distribution (n-gram if seen else unigram)
        probs = lm.get(ctx, unigram)
        chars, weights = zip(*probs.items())

        # 5) sample next character
        next_char = random.choices(chars, weights)[0]

        if next_char == END_TOKEN:       # optional early stop
            break

        generated.append(next_char)

    text = "".join(generated)
    ########################################
    return text

## Part 6
Play with your generate function, try to generate different texts in different language and various values of *n*. No need to submit anything of that.

In [21]:
print(generate('en', 1, "I am", 10, 5))
print(generate('en', 2, "I am", 10, 5))
print(generate('en', 3, "I am", 10, 5))
print(generate('en', 4, "I am ", 10, 5))
print(generate('es', 2, "Soy", 10, 5))
print(generate('es', 3, "Soy", 10, 5))
print(generate('fr', 2, "Je suis", 10, 5))
print(generate('fr', 3, "Je suis", 10, 5))

I amVpsoponkoa
I ama soigBlo 
I ama co/on of
I am I cospeach
Soy p
Soy pos https
Je suis voui
Je suis vouvanLuc


# Testing

Copy the content of the **tests.py** file from the repo and paste below. This will create the results.json file and download it to your machine.

In [22]:
########################################
# PLACE TESTS HERE #
# Create tests
def test_preprocess():
    return {
        'vocab_length': len(preprocess()),
    }

def test_build_lm():
    return {
        'english_2_gram_length': len(build_lm('en', 2, True)),
        'english_3_gram_length': len(build_lm('en', 3, True)),
        'french_3_gram_length': len(build_lm('fr', 3, True)),
        'spanish_3_gram_length': len(build_lm('es', 3, True)),
    }

def test_eval():
    lm = build_lm('en', 3, True)
    return {
        'en_on_en': round(eval(lm, 'en', 3), 2),
        'en_on_fr': round(eval(lm, 'fr', 3), 2),
        'en_on_tl': round(eval(lm, 'tl', 3), 2),
        'en_on_nl': round(eval(lm, 'nl', 3), 2),
    }

def test_match():
    df = match()
    return {
        'df_shape': df.shape,
        'en_en_3': df[(df['source'] == 'en') & (df['target'] == 'en') & (df['n'] == 3)]['perplexity'].values[0],
        'en_tl_3': df[(df['source'] == 'en') & (df['target'] == 'tl') & (df['n'] == 3)]['perplexity'].values[0],
        'en_nl_3': df[(df['source'] == 'en') & (df['target'] == 'nl') & (df['n'] == 3)]['perplexity'].values[0],
    }

def test_generate():
    return {
        'english_2_gram': generate('en', 2, "I am", 20, 5),
        'english_3_gram': generate('en', 3, "I am", 20, 5),
        'english_4_gram': generate('en', 4, "I Love", 20, 5),
        'spanish_2_gram': generate('es', 2, "Soy", 20, 5),
        'spanish_3_gram': generate('es', 3, "Soy", 20, 5),
        'french_2_gram': generate('fr', 2, "Je suis", 20, 5),
        'french_3_gram': generate('fr', 3, "Je suis", 20, 5),
    }

TESTS = [test_preprocess, test_build_lm, test_eval, test_match, test_generate]

# Run tests and save results
res = {}
for test in TESTS:
    try:
        cur_res = test()
        res.update({test.__name__: cur_res})
    except Exception as e:
        res.update({test.__name__: repr(e)})

with open('results.json', 'w') as f:
    json.dump(res, f, indent=2)

# Download the results.json file
files.download('results.json')
########################################

Perplexity evaluations:  48%|████▊     | 122/256 [14:23<16:14,  7.27s/it]

Perplexity evaluations: 100%|██████████| 256/256 [30:05<00:00,  7.05s/it]


NameError: name 'files' is not defined

In [ ]:
# # Show the local files, results.json should be there now and
# # also downloaded to your local machine
# !ls -l

total 12
drwxr-xr-x 2 root root 4096 Mar 27 16:54 data
-rw-r--r-- 1 root root  903 Mar 27 17:05 results.json
drwxr-xr-x 1 root root 4096 Mar 24 13:34 sample_data
